In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import wandb

# Initialize wandb
wandb.init(project="reverse_LRP_mnist")

# Define the neural network architecture
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Convolution 1
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2, 2)(x)
        
        # Convolution 2
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2, 2)(x)

        # Flatten the output for the dense layer
        x = x.view(-1, 64 * 7 * 7)
        
        # Dense layer
        x = self.fc1(x)
        x = nn.ReLU()(x)
        
        # Output layer
        x = self.fc2(x)
        return x

# Load and transform the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Initialize the network and optimizer
model = SimpleNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    wandb.log({"Test Accuracy": accuracy, "Test Loss": test_loss})
    return accuracy

# Training the model with early stopping
def train(model, device, train_loader, optimizer, epoch, target_accuracy=99.0):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        wandb.log({"Train Loss": loss.item()})

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    
            accuracy = test(model, device, test_loader)
            wandb.log({"Test Accuracy": accuracy})
            if accuracy >= target_accuracy:
                print(f"Stopping early: Reached {accuracy:.2f}% accuracy")
                return True
    return False

# Run training
for epoch in range(1, 11):  # 10
    if train(model, device, train_loader, optimizer, epoch):
        break
    
wandb.finish()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.319392
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.069452
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.396833
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.589601
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.311520
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.316898
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.329788
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.185765
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.217727
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.202937
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.158754
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.154318
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.163675
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.268171
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.241389
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.104935
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.142075
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.144865
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.132237
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.117009
Train Epoch: 1 [12800/60000 (

Test Accuracy,▁▆▇▇▇███████████████████████████████████
Test Loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,█▅▂▂▂▂▁▂▁▁▂▁▂▁▁▂▁▁▁▁▂▂▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁
Test Accuracy,99.01
Test Loss,3e-05
Train Loss,0.03693
